# Environement Setup

In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


☁️ Colab rilevato.
Mounted at /content/drive
Cloning into 'MistakeDetection'...
remote: Enumerating objects: 477, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 477 (delta 147), reused 174 (delta 69), pack-reused 215 (from 1)
Receiving objects: 100% (477/477), 50.07 MiB | 33.68 MiB/s, done.
Resolving deltas: 100% (228/228), done.


# Feature Extractor Setup

In [2]:
import os

# CONFIGURAZIONE NOME CARTELLA
DIR_NAME = 'captain_cook_4d_gopro_resized'

if IS_COLAB:
    # --- COLAB (Usa ancora lo zip per velocità) ---
    ZIP_VIDEOS = '/content/drive/MyDrive/MistakeDetection/captain_cook_4d_gopro_resized/captain_cook_4d_gopro_resized.zip'
    DATA_ROOT = os.path.join(ROOT_DIR, "/data")
    LOCAL_VIDEO_DIR = os.path.join(DATA_ROOT, DIR_NAME)

    if not os.path.exists(LOCAL_VIDEO_DIR):
        print(f"☁️ Colab: Estraggo video da {ZIP_VIDEOS}...")
        !mkdir -p {LOCAL_VIDEO_DIR}
        !unzip -q "{ZIP_VIDEOS}" -d "{LOCAL_VIDEO_DIR}"
else:
    # --- LOCALE (Usa la cartella già presente) ---
    DATA_ROOT = os.path.join(ROOT_DIR, 'data')
    LOCAL_VIDEO_DIR = os.path.join(DATA_ROOT, DIR_NAME)

    print(f"Cerco video in: {LOCAL_VIDEO_DIR}")

if os.path.exists(LOCAL_VIDEO_DIR):
    # Conta quanti mp4 ci sono (cerca anche nelle sottocartelle)
    num_videos = 0
    for root, dirs, files in os.walk(LOCAL_VIDEO_DIR):
        for file in files:
            if file.endswith('.mp4'):
                num_videos += 1

    if num_videos > 0:
        print(f"✅ Trovati {num_videos} video .mp4 pronti per l'estrazione.")
    else:
        print(f"⚠️ ATTENZIONE: La cartella '{LOCAL_VIDEO_DIR}' esiste ma NON contiene file .mp4!")
else:
    print(f"❌ ERRORE: La cartella '{LOCAL_VIDEO_DIR}' non esiste.")
    print(f"Assicurati di aver messo la tua cartella '{DIR_NAME}' dentro '{DATA_ROOT}'.")

☁️ Colab: Estraggo video da /content/drive/MyDrive/MistakeDetection/captain_cook_4d_gopro_resized/captain_cook_4d_gopro_resized.zip...
✅ Trovati 384 video .mp4 pronti per l'estrazione.


In [ ]:
!pip install decord

from utils.feature_extractor import PerceptionFeatureExtractor, ExtractionConfig

# 1. Configurazione
config = ExtractionConfig(
    video_dir = LOCAL_VIDEO_DIR,   # Cartella video scompattati
    output_root = DATA_ROOT,       # Dove salvare (creerà /perception_encoder/segment/1s)
    model_name = 'vit_pe_core_base_patch16_224.fb',
    batch_size = 64                # Riduci a 32 se la GPU va in errore (OOM)
)

# 2. Esecuzione
print("Avvio Perception Encoder Feature Extraction...")
extractor = PerceptionFeatureExtractor(config)
extractor.run()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 93.2 MB/s eta 0:00:00
Avvio Perception Encoder Feature Extraction...
Caricamento Modello: vit_pe_core_base_patch16_224.fb...


model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

✅ Modello pronto.
Directory Output configurata: /data/perception_encoder/segment/1s


Estrazione Feature:   0%|          | 0/384 [00:00<?, ?it/s]

In [ ]:
import shutil
import os

ZIP_NAME = 'features_perception_encoder'

if IS_COLAB:
    # --- COLAB: SALVA SU DRIVE ---
    DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/AML_MistakeDetection_DATA'
    OUTPUT_FULL_PATH = os.path.join(DRIVE_OUTPUT_DIR, ZIP_NAME + '.zip')

    print(f"☁️ Colab: Compressione e upload su Drive: {OUTPUT_FULL_PATH}")

    %cd {DATA_ROOT}
    !zip -r -q "{OUTPUT_FULL_PATH}" perception_encoder/

    print("✅ Zip salvato su Drive. Pronto per il training.")

else:
    # --- LOCALE: SALVA IN CARTELLA /DATA ---
    LOCAL_OUTPUT_DIR = os.path.join(ROOT_DIR, 'data')
    os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)

    OUTPUT_BASE_PATH = os.path.join(LOCAL_OUTPUT_DIR, ZIP_NAME)

    print(f"🖥️ Locale: Compressione in corso verso: {LOCAL_OUTPUT_DIR} ...")

    shutil.make_archive(
        base_name=OUTPUT_BASE_PATH,
        format='zip',
        root_dir=DATA_ROOT,
        base_dir='perception_encoder'
    )

    print(f"✅ Zip creato con successo: {OUTPUT_BASE_PATH}.zip")

🖥️ Locale: Compressione in corso verso: c:\Users\enric\Desktop\MistakeDetection\output ...
✅ Zip creato con successo: c:\Users\enric\Desktop\MistakeDetection\output\features_perception_encoder.zip
